In [ ]:
import pandas as pd 

from sklearn.datasets import load_iris as load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from matplotlib.pyplot import plot, show

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [ ]:
# train = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
# test = pd.read_csv('./house-prices-advanced-regression-techniques/test.csv')
from google.colab import files
uploaded = files.upload()
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

Saving test.csv to test (3).csv
Saving train.csv to train (3).csv


In [ ]:
class feature_eng(dict):
    def __init__(self, data):
        self.data = data
        
    def show_columns_with_null(self):
        lst , counter = [] , 1
        for i in self.data.columns:
            if data[i].isna().sum():
                lst.append(i)
                print(f'{counter}. {i} - {self.data[i].dtype} : {self.data[i].isna().sum()} nulls')
                print('----------------------------------------')
                counter += 1
        print(f'total columns with null: {len(lst)}')
        return lst
    
    def show_columns_are_ojebct(self):
        lst , counter = [] , 1
        for i in self.data.columns:
            if self.data[i].dtype == 'object':
                lst.append(i)
                print(f'{counter}.  {i} - {self.data[i].dtype} ')
                print(f"     number of nulls: {self.data[i].isna().sum()} , number of elements: {len(self.data[i].value_counts())}")
                print('----------------------------------------')
                counter += 1
        print(f'tatol object columns : {len(lst)}')
        return lst
    
    def fill_null(self,field):
        if self.data[field].dtype == 'object':
            self.data[field].fillna('*',inplace=True)
        elif input(f"field type is {self.data[field].dtype}, do u want to fill with median?") == "yes":
            self.data[field].fillna(self.data[field].median(), inplace=True) 
            print(f'median of {field} column is {self.data[field].median()}')
        else:
            return self.data
            
    # def make_one_hot(self, field):
    #     temp = pd.get_dummies(self.data[field], prefix=field)
    #     self.data.drop(field, axis=1, inplace=True, errors='ignore')   
    #     return pd.concat([self.data, temp], axis=1)
    
    def labelencode(self,field):
        if self.data[field].dtype != 'object':
            print(f'type of this field is {self.data[field].dtype}')
            return self.data
        print(self.data[field].value_counts())
        print(f'number of titles : {len(self.data[field].value_counts())}')
        self.data[field] = LabelEncoder().fit_transform(data.Street) if input('do you want to proceed?') == 'yes' else self.data[field] 
        return self.data
    
    def labelencode_ordering(self,field):
        pass
    
    def impute(self):
        self.data = IterativeImputer().fit_transform(self.data)
        return self.data
        

In [ ]:
# train.describe()
# train.info()
# test.info()
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [ ]:
y_train = train['SalePrice']
id_test = test['Id']
x_train = train.drop(['SalePrice'], axis=1, errors='ignore', inplace=False)
data_row = pd.concat([x_train, test])
data = data_row.drop('Id', axis=1, errors='ignore', inplace=False)
data.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732.0,Unf,0.0,64.0,796.0,GasA,Ex,Y,SBrkr,796,566,0,1362,1.0,0.0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2.0,480.0,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369.0,Unf,0.0,317.0,1686.0,GasA,Ex,Y,SBrkr,1694,0,0,1694,1.0,0.0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2.0,636.0,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859.0,BLQ,32.0,216.0,1107.0,GasA,Ex,Y,SBrkr,1107,983,0,2090,1.0,0.0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2.0,484.0,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,952.0,952.0,GasA,Gd,Y,FuseF,1022,752,0,1774,0.0,0.0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2.0,468.0,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,

In [ ]:
ins = feature_eng(data)
object_list = ins.show_columns_are_ojebct()

1.  MSZoning - object 
     number of nulls: 4 , number of elements: 5
----------------------------------------
2.  Street - object 
     number of nulls: 0 , number of elements: 2
----------------------------------------
3.  Alley - object 
     number of nulls: 2721 , number of elements: 2
----------------------------------------
4.  LotShape - object 
     number of nulls: 0 , number of elements: 4
----------------------------------------
5.  LandContour - object 
     number of nulls: 0 , number of elements: 4
----------------------------------------
6.  Utilities - object 
     number of nulls: 2 , number of elements: 2
----------------------------------------
7.  LotConfig - object 
     number of nulls: 0 , number of elements: 5
----------------------------------------
8.  LandSlope - object 
     number of nulls: 0 , number of elements: 3
----------------------------------------
9.  Neighborhood - object 
     number of nulls: 0 , number of elements: 25
------------------------

In [ ]:
for i in object_list:
    ins.fill_null(i)
ins.data.head(10)
ins.data.shape

(2919, 79)

In [ ]:
def make_one_hot(data, field):
    temp = pd.get_dummies(data[field], prefix=field)
    data.drop(field, axis=1, inplace=True)
    return pd.concat([data, temp], axis=1)

In [ ]:
for i in object_list:
  data = make_one_hot(data, i)

In [ ]:
data.shape

(2919, 311)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

data = IterativeImputer().fit_transform(data)
data = scale(data)

In [ ]:
x_train, x_test = data[:len(x_train)], data[len(x_train):]
x_train.shape

(1460, 311)

In [ ]:
model = Sequential()
rows, cols = data.shape
model.add(Dense(1, input_dim=cols))
model.compile(loss='mse', optimizer=SGD(lr=0.001))

In [ ]:
history = model.fit(x_train, y_train, epochs=20)
print(mean_absolute_error(model.predict(x_train), y_train)/ y_train.mean())

Epoch 1/20
46/46 [==============================] - 0s 898us/step - loss: 537645824.0000
Epoch 2/20
46/46 [==============================] - 0s 881us/step - loss: 530906272.0000
Epoch 3/20
46/46 [==============================] - 0s 899us/step - loss: 523792832.0000
Epoch 4/20
46/46 [==============================] - 0s 908us/step - loss: 517176256.0000
Epoch 5/20
46/46 [==============================] - 0s 850us/step - loss: 512345280.0000
Epoch 6/20
46/46 [==============================] - 0s 952us/step - loss: 510120288.0000
Epoch 7/20
46/46 [==============================] - 0s 923us/step - loss: 504052224.0000
Epoch 8/20
46/46 [==============================] - 0s 929us/step - loss: 500145984.0000
Epoch 9/20
46/46 [==============================] - 0s 952us/step - loss: 496535040.0000
Epoch 10/20
46/46 [==============================] - 0s 1ms/step - loss: 493162272.0000
Epoch 11/20
46/46 [==============================] - 0s 1ms/step - loss: 490276160.0000
Epoch 12/20
46/46 [====

In [ ]:
x_train, x_test_total , y_train, y_test_total = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)
x_val, x_ts, y_val, y_ts = train_test_split(x_test_total, y_test_total, test_size=0.5, shuffle=True) 
history = model.fit(x_train, y_train, epochs=50)
print(mean_absolute_error(model.predict(x_val), y_val)/ y_val.mean())
print(mean_absolute_error(model.predict(x_ts), y_ts)/ y_ts.mean())

Epoch 1/50
30/30 [==============================] - 0s 1ms/step - loss: 431718464.0000
Epoch 2/50
30/30 [==============================] - 0s 1ms/step - loss: 428495520.0000
Epoch 3/50
30/30 [==============================] - 0s 1ms/step - loss: 425984064.0000
Epoch 4/50
30/30 [==============================] - 0s 969us/step - loss: 422910592.0000
Epoch 5/50
30/30 [==============================] - 0s 1ms/step - loss: 421763552.0000
Epoch 6/50
30/30 [==============================] - 0s 1ms/step - loss: 419814016.0000
Epoch 7/50
30/30 [==============================] - 0s 1ms/step - loss: 418194656.0000
Epoch 8/50
30/30 [==============================] - 0s 1ms/step - loss: 417086400.0000
Epoch 9/50
30/30 [==============================] - 0s 1ms/step - loss: 413466976.0000
Epoch 10/50
30/30 [==============================] - 0s 1ms/step - loss: 413801056.0000
Epoch 11/50
30/30 [==============================] - 0s 1ms/step - loss: 412550816.0000
Epoch 12/50
30/30 [====================

In [ ]:
predictions = model.predict(x_test)
new = predictions.reshape(-1)
print(new.shape)
df = pd.DataFrame(data=list(zip(id_test, new)), columns=['Id', 'SalePrice'])

from google.colab import files
df.to_csv('submis.csv', index=False) 
files.download('submis.csv')
df.head(2)

(1459,)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Id,SalePrice
0,1461,109666.117188
1,1462,157674.953125
